In [1]:
# | default_exp api_wrappers


In [1]:
# |hide
from nbdev.showdoc import *

In [3]:
# |export

import subprocess

import openai
import time
import re

# API wrappers

> Helper functions that make it easier to use the OpenAI API


## Fine-tune


In [12]:
# |export
def fine_tune(train_file, valid_file, model: str = "ada"):
    # run the fine tuning
    result = subprocess.run(
        f"openai api fine_tunes.create -t {train_file} -v {valid_file} -m {model}",
        shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    modelname = re.findall(r'completions.create -m ([\w\d:-]+) -p', result.stdout)[0]
    # sync runs with wandb
    subprocess.run("openai wandb sync -n 1", shell=True)
    return modelname

In [4]:
_stdout_fragment = "openai api completions.create -m ada:ft-epfl-2022-06-23-09-10-58 -p <YOUR_PROMPT>"

## Predict


In [5]:
# |export
def query_gpt3(model, df, temperature=0, max_tokens=10):
    completions = []
    for i, row in df.iterrows():
        completion = openai.Completion.create(
            model=model,
            prompt=row["prompt"],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        completions.append(completion)
        time.sleep(5)

    return completions

In [1]:
# |export
def extract_prediction(completion):
    return completion["choices"][0]["text"].split("@")[0].strip()
